# Get the Data and Perform Data Cleaning

In [ ]:
import pandas as pd

crime_df = pd.read_csv("data/nyc_crime_2015_to_recent.csv")
rent_df = pd.read_csv("data/medianAskingRent.csv")
zip_df = pd.read_csv("data/nyc_zipcodes.csv")

crime_df_copy = crime_df.copy()
rent_df_copy = rent_df.copy()
zip_df_copy = zip_df.copy()

print("\nZip Data Preview:")
print(zip_df.head()); print('\n')

print("Crime Data Preview:")
print(crime_df.head()); print('\n')

print("\nRent Data Preview:")
print(rent_df.head()); print('\n')

In [ ]:
print(crime_df.info()); print('\n')
print('Length of crime_df: ', len(crime_df)); print('\n')
crime_df = crime_df.dropna()
print('Length of crime_df after dropping NA values: ', len(crime_df)); print('\n')


In [ ]:
# Turn to long format
rent_long_df = rent_df.melt(
    id_vars=["areaName", "Borough", "areaType"],  # fix column
    var_name="date",        # original column name → 'date'
    value_name="median_rent"  # original column value → 'median_rent'
)

rent_long_df["date"] = pd.to_datetime(rent_long_df["date"], format="%Y-%m")

rent_long_df["Borough"] = rent_long_df["Borough"].str.upper()

print("\n📅 Transformed Rent Data:")
print(rent_long_df.head()); print('\n')
print(rent_long_df.tail()); print('\n')
print('Length of rent_long_df: ', len(rent_long_df)); print('\n')
rent_long_df = rent_long_df.dropna()
print('Length of rent_long_df after dropping NA values: ', len(rent_long_df))


In [ ]:
print("🔍 Rent Area Name (unique):")
print(len(rent_long_df["areaName"].unique()))
rent_areas = rent_long_df["areaName"].unique().tolist()
print("\n🔍 Crime Precinct Area (unique):")
print(len(crime_df["precinct_area"].unique()))
crime_areas = crime_df["precinct_area"].unique().tolist()
print("\n🔍 Zip Neighborhood (unique):")
print(len(zip_df["Neighborhood"].unique()))
zip_areas = zip_df["Neighborhood"].unique().tolist()
# pd.DataFrame(rent_areas, columns=["Rent Area"]).to_csv("rent_area_list.csv", index=False)
# pd.DataFrame(crime_areas, columns=["Crime Area"]).to_csv("crime_area_list.csv", index=False)


# Data Mapping


In [ ]:
import pandas as pd

# Your existing mappings
precinct_to_rent_mapping = {
    "financial district / tribeca": "All Downtown",
    "chinatown / lower east side": "All Lower East Side",
    "greenwich village / west village": "West Village",
    "lower east side": "All Lower East Side",
    "east village / alphabet city": "East Village",
    "chelsea / hudson yards": "Chelsea",
    "flatiron / midtown south": "Flatiron",
    "theater district / midtown": "All Midtown",
    "midtown east": "Midtown East",
    "columbus circle / midtown west": "Midtown West",
    "upper east side": "Upper East Side",
    "upper west side": "Upper West Side",
    "central park": "Central Park South",
    "east harlem south": "East Harlem",
    "upper west side north": "All Upper West Side",
    "east harlem north": "East Harlem",
    "morningside heights / manhattanville": "Morningside Heights",
    "west harlem": "West Harlem",
    "hamilton heights / sugar hill": "Hamilton Heights",
    "east harlem central": "East Harlem",
    "washington heights south": "Washington Heights",
    "washington heights north": "Washington Heights",
    "south bronx / mott haven": "Mott Haven",
    "hunts point": "Hunts Point",
    "morrisania / crotona park east": "Crotona Park East",
    "soundview": "Soundview",
    "highbridge / grand concourse": "Highbridge",
    "throgs neck / co-op city": "Throgs Neck",
    "university heights / fordham": "University Heights",
    "woodlawn / wakefield": "Wakefield",
    "belmont / tremont": "Belmont",
    "pelham parkway / morris park": "Pelham Parkway",
    "riverdale / kingsbridge": "Riverdale",
    "norwood / bedford park": "Bedford Park",
    "coney island / brighton beach": "Coney Island",
    "sheepshead bay / madison": "Sheepshead Bay",
    "bensonhurst": "Bensonhurst",
    "marine park / mill basin": "Marine Park",
    "borough park": "Borough Park",
    "east flatbush": "East Flatbush",
    "bay ridge / dyker heights": "Bay Ridge",
    "canarsie": "Canarsie",
    "flatbush / ditmas park": "Flatbush",
    "crown heights north": "Crown Heights",
    "sunset park / windsor terrace": "Windsor Terrace",
    "brownsville": "Brownsville",
    "east new york": "East New York",
    "red hook / carroll gardens": "Carroll Gardens",
    "crown heights south": "Crown Heights",
    "park slope": "Park Slope",
    "bedford-stuyvesant north": "Bedford-Stuyvesant",
    "bedford-stuyvesant central": "Bedford-Stuyvesant",
    "bushwick": "Bushwick",
    "brooklyn heights / downtown brooklyn": "Downtown Brooklyn",
    "fort greene / clinton hill": "Fort Greene",
    "williamsburg south": "Williamsburg",
    "greenpoint": "Greenpoint",
    "rockaway beach": "Rockaway All",
    "far rockaway": "Rockaway All",
    "richmond hill / woodhaven": "Richmond Hill",
    "jamaica central": "Jamaica",
    "ridgewood / glendale": "Ridgewood",
    "queens village": "Queens Village",
    "ozone park / howard beach": "Ozone Park",
    "fresh meadows / hillcrest": "Fresh Meadows",
    "long island city": "Long Island City",
    "flushing": "Flushing",
    "elmhurst / corona": "Elmhurst",
    "bayside": "Bayside",
    "forest hills / rego park": "Forest Hills",
    "jamaica south": "Jamaica",
    "astoria": "Astoria",
    "jackson heights / east elmhurst": "Jackson Heights",
    "st. george / stapleton": "Staten Island",
    "willowbrook / mid-island": "Staten Island",
    "tottenville / great kills": "Staten Island",
    "south shore": "Staten Island"
}

rent_to_zipcode_neighborhood = {
    "All Downtown": "Lower Manhattan",
    "All Lower East Side": "Lower East Side",
    "All Midtown": "Chelsea and Clinton",
    "All Upper East Side": "Upper East Side",
    "All Upper West Side": "Upper West Side",
    "All Upper Manhattan": "Central Harlem",
    "Battery Park City": "Lower Manhattan",
    "Central Harlem": "Central Harlem",
    "Central Park South": "Upper East Side",
    "Chelsea": "Chelsea and Clinton",
    "Chinatown": "Lower East Side",
    "East Harlem": "East Harlem",
    "East Village": "Lower East Side",
    "Financial District": "Lower Manhattan",
    "Flatiron": "Gramercy Park and Murray Hill",
    "Gramercy Park": "Gramercy Park and Murray Hill",
    "Greenwich Village": "Greenwich Village and Soho",
    "Hamilton Heights": "Central Harlem",
    "Inwood": "Inwood and Washington Heights",
    "Little Italy": "Greenwich Village and Soho",
    "Lower East Side": "Lower East Side",
    "Midtown": "Chelsea and Clinton",
    "Midtown East": "Gramercy Park and Murray Hill",
    "Midtown South": "Chelsea and Clinton",
    "Midtown West": "Chelsea and Clinton",
    "Morningside Heights": "Central Harlem",
    "Nolita": "Greenwich Village and Soho",
    "Roosevelt Island": "Upper East Side",
    "Soho": "Greenwich Village and Soho",
    "Stuyvesant Town/PCV": "Lower East Side",
    "Tribeca": "Lower Manhattan",
    "Upper East Side": "Upper East Side",
    "Upper West Side": "Upper West Side",
    "Washington Heights": "Inwood and Washington Heights",
    "West Harlem": "Central Harlem",
    "West Village": "Greenwich Village and Soho",
    "Manhattan": "Manhattan",
    
    # Brooklyn
    "Bay Ridge": "Southwest Brooklyn",
    "Bedford-Stuyvesant": "Central Brooklyn",
    "Boerum Hill": "Northwest Brooklyn",
    "Brooklyn": "Brooklyn",
    "Brooklyn Heights": "Northwest Brooklyn",
    "Bushwick": "Bushwick and Williamsburg",
    "Carroll Gardens": "Northwest Brooklyn",
    "Clinton Hill": "Northwest Brooklyn",
    "Cobble Hill": "Northwest Brooklyn",
    "Columbia St Waterfront District": "Northwest Brooklyn",
    "Crown Heights": "Central Brooklyn",
    "DUMBO": "Northwest Brooklyn",
    "Ditmas Park": "Flatbush",
    "Downtown Brooklyn": "Northwest Brooklyn",
    "East Brooklyn": "East New York and New Lots",
    "East Flatbush": "Flatbush",
    "East New York": "East New York and New Lots",
    "Flatbush": "Flatbush",
    "Fort Greene": "Northwest Brooklyn",
    "Gowanus": "Northwest Brooklyn",
    "Greenpoint": "Greenpoint",
    "Greenwood": "Northwest Brooklyn",
    "Kensington": "Borough Park",
    "North Brooklyn": "Bushwick and Williamsburg",
    "Northwest Brooklyn": "Northwest Brooklyn",
    "Park Slope": "Northwest Brooklyn",
    "Prospect Heights": "Northwest Brooklyn",
    "Prospect Lefferts Gardens": "Flatbush",
    "Prospect Park": "Northwest Brooklyn",
    "Prospect Park South": "Flatbush",
    "Red Hook": "Northwest Brooklyn",
    "Sheepshead Bay": "Southern Brooklyn",
    "South Brooklyn": "Southwest Brooklyn",
    "Sunset Park": "Sunset Park",
    "Williamsburg": "Bushwick and Williamsburg",
    "Windsor Terrace": "Northwest Brooklyn",
    "Borough Park": "Borough Park",
    "Brighton Beach": "Southern Brooklyn",
    "Bensonhurst": "Borough Park",
    "Canarsie": "Canarsie and Flatlands",
    "Bath Beach": "Southwest Brooklyn",
    "Brownsville": "Central Brooklyn",
    "Coney Island": "Southern Brooklyn",
    "Marine Park": "Canarsie and Flatlands",
    "Bergen Beach": "Canarsie and Flatlands",
    "Flatlands": "Canarsie and Flatlands",
    "Dyker Heights": "Southwest Brooklyn",
    "Mill Basin": "Canarsie and Flatlands",
    "Manhattan Beach": "Southern Brooklyn",
    
    # Bronx
    "Bronx": "Bronx",
    "Kingsbridge": "Kingsbridge and Riverdale",
    "Riverdale": "Kingsbridge and Riverdale",
    "University Heights": "High Bridge and Morrisania",
    "Concourse": "High Bridge and Morrisania",
    "Highbridge": "High Bridge and Morrisania",
    "Mott Haven": "Hunts Point and Mott Haven",
    "Parkchester": "Southeast Bronx",
    "Bedford Park": "Bronx Park and Fordham",
    "Melrose": "High Bridge and Morrisania",
    "Belmont": "Bronx Park and Fordham",
    "Fordham": "Bronx Park and Fordham",
    "Norwood": "Bronx Park and Fordham",
    "Morris Heights": "High Bridge and Morrisania",
    "Tremont": "Central Bronx",
    "Pelham Parkway": "Southeast Bronx",
    "Soundview": "Southeast Bronx",
    "Woodstock": "Hunts Point and Mott Haven",
    "Bronxwood": "Northeast Bronx",
    "Castle Hill": "Southeast Bronx",
    "Pelham Bay": "Southeast Bronx",
    "Throgs Neck": "Southeast Bronx",
    "Morrisania": "High Bridge and Morrisania",
    "Marble Hill": "Kingsbridge and Riverdale",
    "Van Nest": "Southeast Bronx",
    "Pelham Gardens": "Northeast Bronx",
    "Longwood": "Hunts Point and Mott Haven",
    "Woodlawn": "Northeast Bronx",
    "Hunts Point": "Hunts Point and Mott Haven",
    "Crotona Park East": "Central Bronx",
    "Edenwald": "Northeast Bronx",
    "Wakefield": "Northeast Bronx",
    "Williamsbridge": "Northeast Bronx",
    "East Tremont": "Central Bronx",
    "Morris Park": "Southeast Bronx",
    
    # Queens
    "Astoria": "Northwest Queens",
    "Bayside": "Northeast Queens",
    "Corona": "West Queens",
    "Elmhurst": "West Queens",
    "Flushing": "North Queens",
    "Forest Hills": "West Central Queens",
    "Jackson Heights": "West Queens",
    "Jamaica": "Jamaica",
    "Kew Gardens": "West Central Queens",
    "Long Island City": "Northwest Queens",
    "Northeast Queens": "Northeast Queens",
    "Northwest Queens": "Northwest Queens",
    "Queens": "Queens",
    "Rego Park": "West Central Queens",
    "Sunnyside": "Northwest Queens",
    "Woodside": "West Queens",
    "Briarwood": "Jamaica",
    "Jamaica Estates": "Jamaica",
    "Ridgewood": "West Central Queens",
    "Whitestone": "North Queens",
    "Glendale": "West Central Queens",
    "Maspeth": "West Queens",
    "Douglaston": "Northeast Queens",
    "Richmond Hill": "Southwest Queens",
    "East Elmhurst": "West Queens",
    "North Corona": "West Queens",
    "Bellerose": "Southeast Queens",
    "Middle Village": "West Central Queens",
    "Woodhaven": "Southwest Queens",
    "Oakland Gardens": "Northeast Queens",
    "Queens Village": "Southeast Queens",
    "Howard Beach": "Southwest Queens",
    "St. Albans": "Southeast Queens",
    "Pomonok": "Central Queens",
    "College Point": "North Queens",
    "Jamaica Hills": "Jamaica",
    "Fresh Meadows": "Central Queens",
    "Hillcrest": "Central Queens",
    "Little Neck": "Northeast Queens",
    "Ozone Park": "Southwest Queens",
    "Laurelton": "Southeast Queens",
    "Clearview": "North Queens",
    "Hollis": "Southeast Queens",
    "South Richmond Hill": "Southwest Queens",
    "South Jamaica": "Jamaica",
    "South Ozone Park": "Southwest Queens",
    "Springfield Gardens": "Southeast Queens",
    "Rockaway All": "Rockaways",
    "The Rockaways": "Rockaways",
    "Kew Gardens Hills": "Central Queens",
    
    # Staten Island
    "Staten Island": "Staten Island"
}

rent_to_precinct = {
    # Manhattan 
    "All Downtown": "1,5,6,7",
    "All Lower East Side": "5,7,9",
    "All Midtown": "10,13,14,17,18",
    "All Upper East Side": "19,23",
    "All Upper West Side": "20,24",
    "All Upper Manhattan": "26,28,30,32,33,34",
    "Battery Park City": "1",
    "Central Harlem": "28,32",
    "Central Park South": "19",
    "Chelsea": "10",
    "Chinatown": "5",
    "East Harlem": "23,25",
    "East Village": "9",
    "Financial District": "1",
    "Flatiron": "13",
    "Gramercy Park": "13",
    "Greenwich Village": "6",
    "Hamilton Heights": "30,32",
    "Inwood": "34",
    "Little Italy": "5",
    "Lower East Side": "7,9",
    "Midtown": "14,18",
    "Midtown East": "17",
    "Midtown South": "13",
    "Midtown West": "10,14",
    "Morningside Heights": "26",
    "Nolita": "5",
    "Roosevelt Island": "19",
    "Soho": "1,5",
    "Stuyvesant Town/PCV": "13",
    "Tribeca": "1",
    "Upper East Side": "19,23",
    "Upper West Side": "20,24",
    "Washington Heights": "33,34",
    "West Harlem": "26,30",
    "West Village": "6",
    "Manhattan": "1,5,6,7,9,10,13,14,17,18,19,20,23,24,25,26,28,30,32,33,34",
    
    # Brooklyn
    "Bay Ridge": "68",
    "Bedford-Stuyvesant": "79,81",
    "Boerum Hill": "84",
    "Brooklyn": "60-94",
    "Brooklyn Heights": "84",
    "Bushwick": "83",
    "Carroll Gardens": "76",
    "Clinton Hill": "88",
    "Cobble Hill": "76",
    "Columbia St Waterfront District": "76",
    "Crown Heights": "71,77",
    "DUMBO": "84",
    "Ditmas Park": "70",
    "Downtown Brooklyn": "84",
    "East Brooklyn": "75",
    "East Flatbush": "67",
    "East New York": "75",
    "Flatbush": "70",
    "Fort Greene": "88",
    "Gowanus": "78",
    "Greenpoint": "94",
    "Greenwood": "72",
    "Kensington": "66",
    "North Brooklyn": "90,94",
    "Northwest Brooklyn": "76,78,84,88",
    "Park Slope": "78",
    "Prospect Heights": "77",
    "Prospect Lefferts Gardens": "71",
    "Prospect Park": "78",
    "Prospect Park South": "70",
    "Red Hook": "76",
    "Sheepshead Bay": "61",
    "South Brooklyn": "62,68",
    "Sunset Park": "72",
    "Williamsburg": "90,94",
    "Windsor Terrace": "72",
    "Borough Park": "66",
    "Brighton Beach": "60",
    "Bensonhurst": "62",
    "Canarsie": "69",
    "Bath Beach": "62",
    "Brownsville": "73",
    "Coney Island": "60",
    "Marine Park": "63",
    "Bergen Beach": "63",
    "Flatlands": "63",
    "Dyker Heights": "68",
    "Mill Basin": "63",
    "Manhattan Beach": "61",
    
    # Bronx
    "Bronx": "40-52",
    "Kingsbridge": "50",
    "Riverdale": "50",
    "University Heights": "46",
    "Concourse": "44",
    "Highbridge": "44",
    "Mott Haven": "40",
    "Parkchester": "43",
    "Bedford Park": "52",
    "Melrose": "40",
    "Belmont": "48",
    "Fordham": "46,48",
    "Norwood": "52",
    "Morris Heights": "46",
    "Tremont": "48",
    "Pelham Parkway": "49",
    "Soundview": "43",
    "Woodstock": "40",
    "Bronxwood": "47",
    "Castle Hill": "43",
    "Pelham Bay": "45",
    "Throgs Neck": "45",
    "Morrisania": "42",
    "Marble Hill": "50",
    "Van Nest": "49",
    "Pelham Gardens": "49",
    "Longwood": "41",
    "Woodlawn": "47",
    "Hunts Point": "41",
    "Crotona Park East": "42",
    "Edenwald": "47",
    "Wakefield": "47",
    "Williamsbridge": "47",
    "East Tremont": "48",
    "Morris Park": "49",
    
    # Queens
    "Astoria": "114",
    "Bayside": "111",
    "Corona": "110,115",
    "Elmhurst": "110,115",
    "Flushing": "109,111",
    "Forest Hills": "112",
    "Jackson Heights": "115",
    "Jamaica": "103,113",
    "Kew Gardens": "102",
    "Long Island City": "108,114",
    "Northeast Queens": "111",
    "Northwest Queens": "108,114",
    "Queens": "100-116",
    "Rego Park": "112",
    "Sunnyside": "108",
    "Woodside": "108",
    "Briarwood": "107",
    "Jamaica Estates": "107",
    "Ridgewood": "104",
    "Whitestone": "109",
    "Glendale": "104",
    "Maspeth": "104",
    "Douglaston": "111",
    "Richmond Hill": "102",
    "East Elmhurst": "115",
    "North Corona": "115",
    "Bellerose": "105",
    "Middle Village": "104",
    "Woodhaven": "102",
    "Oakland Gardens": "111",
    "Queens Village": "105",
    "Howard Beach": "106",
    "St. Albans": "113",
    "Pomonok": "107",
    "College Point": "109",
    "Jamaica Hills": "107",
    "Fresh Meadows": "107",
    "Hillcrest": "107",
    "Little Neck": "111",
    "Ozone Park": "106",
    "Laurelton": "105",
    "Clearview": "109",
    "Hollis": "103",
    "South Richmond Hill": "106",
    "South Jamaica": "113",
    "South Ozone Park": "106",
    "Springfield Gardens": "105",
    "Rockaway All": "100,101",
    "The Rockaways": "100,101",
    "Kew Gardens Hills": "107",
    
    # Staten Island
    "Staten Island": "120,121,122,123"
}


In [ ]:
# Mapping precinct_area to corresponding rent area name
crime_df["areaName"] = crime_df["precinct_area"].str.lower().map(precinct_to_rent_mapping)
crime_df.head()

In [ ]:
rent_long_df.head()

In [ ]:
# Swith to datetime format
crime_df["month"] = pd.to_datetime(crime_df["month"])
rent_long_df["date"] = pd.to_datetime(rent_long_df["date"])

# Turn crime column: month to 'date'
crime_df = crime_df.rename(columns={"month": "date"})

merged_df = pd.merge(crime_df, rent_long_df, how="inner", on=["areaName", "date"])


merged_df["date"] = pd.to_datetime(merged_df["date"]).dt.strftime("%Y-%m")
merged_df = merged_df.reset_index()

print("Lenght of merged_df: ", len(merged_df))

merged_df.head()


In [ ]:
merged_df.info()

In [ ]:
# Mapping precinct_area to corresponding rent area name
rent_to_zipcode_neighborhood_lower = {
    k.lower(): v for k, v in rent_to_zipcode_neighborhood.items()
}
merged_df["Neighborhood"] = merged_df["areaName"].str.lower().map(rent_to_zipcode_neighborhood_lower)
merged_df.sample()

In [ ]:
neighborhood_to_zip = zip_df.groupby("Neighborhood")["ZIP Codes"].apply(list).to_dict()
print(neighborhood_to_zip)
merged_df["ZIP Codes"] = merged_df["Neighborhood"].map(neighborhood_to_zip)
merged_df.dropna(inplace=True)

print("Shape of merged_df:", merged_df.shape)
merged_df.head()

## Danger Ratio



The **Danger Ratio** is a normalized metric that measures **crime severity relative to housing cost**.

Crime incidents are weighted by severity (**Felony = 3, Misdemeanor = 2, Violation = 1**) to compute a weighted crime score. The Danger Ratio is defined as:

> **weighted crime score ÷ median rent**

This metric enables comparison across neighborhoods with different rent levels, highlighting areas where **higher crime severity is concentrated relative to housing cost**.

In [ ]:
weight_map = {
    "FELONY": 3,
    "MISDEMEANOR": 2,
    "VIOLATION": 1
}

merged_df["crime_weight"] = merged_df["law_cat_cd"].map(weight_map)
merged_df["weighted_crime"] = merged_df["count"] * merged_df["crime_weight"]
merged_df["danger_ratio"] = merged_df["weighted_crime"] / merged_df["median_rent"]

merged_df = merged_df.drop(columns=["crime_weight", "weighted_crime"])

merged_df.head()

## Save Merged Dataframe as CSV

In [ ]:
# Define the target folder and file path
from pathlib import Path
folder_path = Path("data")
folder_path.mkdir(parents=True, exist_ok=True)  # Create the folder if it doesn't exist

csv_path = folder_path / "nyc_rent&crime_2015_to_recent.csv"

# Save DataFrame
merged_df.to_csv(csv_path, index=False)

print(f"File saved to: {csv_path}")

# Exploratory Data Analysis of Each Dataset

## Overall Crime Trend Analysis

In [ ]:
import pandas as pd
import plotly.express as px

# Ensure date is datetime
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Aggregate to MONTHLY total crime
monthly_total_crime = (
    merged_df
    .set_index("date")
    .resample("ME")["count"]
    .sum()
    .reset_index()
)

# Create line plot
fig = px.line(
    monthly_total_crime,
    x="date",
    y="count",
    title="Total Crime Incidents Over Time (Monthly)",
    labels={
        "date": "Date",
        "count": "Total Crime Incidents"
    }
)

# Professional styling
fig.update_traces(
    line=dict(width=2),
    hovertemplate="Month: %{x|%b %Y}<br>Total Crimes: %{y:,}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    margin=dict(l=60, r=40, t=60, b=50)
)

fig.show()
# Export a static image for GitHub rendering
import os
os.makedirs("figs", exist_ok=True)
fig.write_image("figs/fig_01.png", scale=2)
from IPython.display import HTML
HTML(f'<img src="figs/fig_01.png" alt="Figure" width="100%"/>')


## Geographical Crime Distribution Analysis

In [ ]:
area_total_crime = (
    merged_df
    .groupby("areaName", as_index=False)["count"]
    .sum()
)

# Select top 15 areas
top_areas = (
    area_total_crime
    .sort_values(by="count", ascending=False)
    .head(15)
)

fig1 = px.bar(
    top_areas,
    x="count",
    y="areaName",
    orientation="h",
    title="Top 15 Areas by Total Crime Incidents (10-Year Aggregate)",
    labels={
        "count": "Total Crime Incidents",
        "areaName": "Area"
    }
)

fig1.update_layout(
    template="plotly_white",
    yaxis=dict(categoryorder="total ascending"),
    margin=dict(l=120, r=40, t=60, b=50)
)

fig1.update_traces(
    hovertemplate="Area: %{y}<br>Total Crimes: %{x:,}<extra></extra>"
)

fig1.show()

In [ ]:
borough_total_crime = (
    merged_df
    .groupby("Borough", as_index=False)["count"]
    .sum()
)

fig2 = px.bar(
    borough_total_crime,
    x="Borough",
    y="count",
    title="Total Crime Incidents by Borough (10-Year Aggregate)",
    labels={
        "Borough": "Borough",
        "count": "Total Crime Incidents"
    }
)

fig2.update_layout(
    template="plotly_white",
    margin=dict(l=60, r=40, t=60, b=50)
)

fig2.update_traces(
    hovertemplate="Borough: %{x}<br>Total Crimes: %{y:,}<extra></extra>"
)

fig2.show()

## Cross-Analysis of Area and Crime Category

In [ ]:
borough_category_crime = (
    merged_df
    .groupby(["Borough", "law_cat_cd"], as_index=False)["count"]
    .sum()
)

fig = px.bar(
    borough_category_crime,
    x="Borough",
    y="count",
    color="law_cat_cd",
    title="Crime Incidents by Borough and Offense Category (10-Year Aggregate)",
    labels={
        "count": "Total Crime Incidents",
        "law_cat_cd": "Offense Category",
        "Borough": "Borough"
    },
    barmode="stack",
    color_discrete_map={
        "FELONY": "#d62728",        # red
        "MISDEMEANOR": "#ff7f0e",   # orange
        "VIOLATION": "#2ca02c"      # green
    }
)

fig.update_layout(
    template="plotly_white",
    legend_title_text="Offense Category",
    margin=dict(l=60, r=40, t=60, b=50)
)

fig.update_traces(
    hovertemplate=(
        "Borough: %{x}<br>"
        "Category: %{marker.color}<br>"
        "Total Crimes: %{y:,}<extra></extra>"
    )
)

fig.show()
# Export a static image for GitHub rendering
import os
os.makedirs("figs", exist_ok=True)
fig.write_image("figs/fig_02.png", scale=2)
from IPython.display import HTML
HTML(f'<img src="figs/fig_02.png" alt="Figure" width="100%"/>')


## Time Trend Analysis for Specific Area/Borough

In [ ]:
# Ensure date is datetime
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Borough-level Crime Trend
selected_borough = "MANHATTAN"

borough_crime_data = merged_df[merged_df["Borough"] == selected_borough]

borough_monthly_category = (
    borough_crime_data
    .set_index("date")
    .groupby("law_cat_cd")["count"]
    .resample("ME")
    .sum()
    .reset_index()
)

fig_borough = px.line(
    borough_monthly_category,
    x="date",
    y="count",
    color="law_cat_cd",
    title=f"Monthly Crime Trends by Offense Category in {selected_borough}",
    labels={
        "date": "Date",
        "count": "Total Crime Incidents",
        "law_cat_cd": "Offense Category"
    },
    color_discrete_map={
        "FELONY": "#d62728",
        "MISDEMEANOR": "#ff7f0e",
        "VIOLATION": "#2ca02c"
    }
)

fig_borough.update_layout(
    template="plotly_white",
    legend_title_text="Offense Category",
    margin=dict(l=60, r=40, t=60, b=50)
)

fig_borough.update_traces(
    line=dict(width=2),
    hovertemplate=(
        "Month: %{x|%b %Y}<br>"
        "Category: %{marker.color}<br>"
        "Total Crimes: %{y:,}<extra></extra>"
    )
)

fig_borough.show()


In [ ]:
# Area-level Crime Trend
selected_area = "Financial District / Tribeca"

area_crime_data = merged_df[merged_df["precinct_area"] == selected_area]

area_monthly_category = (
    area_crime_data
    .set_index("date")
    .groupby("law_cat_cd")["count"]
    .resample("ME")
    .sum()
    .reset_index()
)

fig_area = px.line(
    area_monthly_category,
    x="date",
    y="count",
    color="law_cat_cd",
    title=f"Monthly Crime Trends by Offense Category in {selected_area}",
    labels={
        "date": "Date",
        "count": "Total Crime Incidents",
        "law_cat_cd": "Offense Category"
    },
    color_discrete_map={
        "FELONY": "#d62728",
        "MISDEMEANOR": "#ff7f0e",
        "VIOLATION": "#2ca02c"
    }
)

fig_area.update_layout(
    template="plotly_white",
    legend_title_text="Offense Category",
    margin=dict(l=60, r=40, t=60, b=50)
)

fig_area.update_traces(
    line=dict(width=2),
    hovertemplate=(
        "Month: %{x|%b %Y}<br>"
        "Category: %{marker.color}<br>"
        "Total Crimes: %{y:,}<extra></extra>"
    )
)

fig_area.show()

## Crime and Rent Correlation Analysis

In [ ]:
# Group by precinct_area (or areaName) to get total crime and median rent per area
# Note: median_rent is per area/month in original data, so group by area first to get an average/total crime over time per area
area_agg_data = merged_df.groupby('precinct_area').agg(
    total_crime=('count', 'sum'), # Calculate total crime over time for each area
    avg_median_rent=('median_rent', 'mean') # Calculate average median rent over time for each area
).reset_index()

# Create a scatter plot to see the relationship between total crime and average median rent
fig = px.scatter(area_agg_data, x='avg_median_rent', y='total_crime',
                 hover_name='precinct_area', # Show area name on hover
                 title='Total Crime vs. Average Median Rent by Precinct Area')
fig.show()

# Optional: Analyze relationship for specific crime categories
felony_by_area = merged_df[merged_df['law_cat_cd'] == 'FELONY'].groupby('precinct_area')['count'].sum().reset_index(name='felony_count')
area_rent_felony = area_agg_data.merge(felony_by_area, on='precinct_area', how='left').fillna(0)

fig2 = px.scatter(area_rent_felony, x='avg_median_rent', y='felony_count',
                 hover_name='precinct_area',
                 title='Felony Count vs. Average Median Rent by Precinct Area')
fig2.show()
# Export a static image for GitHub rendering
import os
os.makedirs("figs", exist_ok=True)
fig.write_image("figs/fig_03.png", scale=2)
from IPython.display import HTML
HTML(f'<img src="figs/fig_03.png" alt="Figure" width="100%"/>')


In [ ]:
# Aggregate crime and rent by precinct area
area_agg_data = (
    merged_df
    .groupby("precinct_area", as_index=False)
    .agg(
        total_crime=("count", "sum"),
        avg_median_rent=("median_rent", "mean")
    )
)

# Total Crime vs. Average Median Rent
fig1 = px.scatter(
    area_agg_data,
    x="avg_median_rent",
    y="total_crime",
    hover_name="precinct_area",
    title="Total Crime Incidents vs. Average Median Rent by Precinct Area",
    labels={
        "avg_median_rent": "Average Median Rent (USD)",
        "total_crime": "Total Crime Incidents"
    }
)

fig1.update_traces(
    marker=dict(size=6, opacity=0.6, line=dict(width=0)),
    hovertemplate=(
        "Area: %{hovertext}<br>"
        "Avg. Rent: $%{x:,.0f}<br>"
        "Total Crimes: %{y:,}<extra></extra>"
    )
)

fig1.update_layout(
    template="plotly_white",
    xaxis=dict(tickprefix="$"),
    margin=dict(l=60, r=40, t=60, b=50)
)

fig1.show()


# Felony Crime vs. Average Median Rent
felony_by_area = (
    merged_df[merged_df["law_cat_cd"] == "FELONY"]
    .groupby("precinct_area", as_index=False)["count"]
    .sum()
    .rename(columns={"count": "felony_count"})
)

area_rent_felony = (
    area_agg_data
    .merge(felony_by_area, on="precinct_area", how="left")
    .fillna({"felony_count": 0})
)

fig2 = px.scatter(
    area_rent_felony,
    x="avg_median_rent",
    y="felony_count",
    hover_name="precinct_area",
    title="Felony Crime Incidents vs. Average Median Rent by Precinct Area",
    labels={
        "avg_median_rent": "Average Median Rent (USD)",
        "felony_count": "Total Felony Incidents"
    }
)

fig2.update_traces(
    marker=dict(size=6, opacity=0.6, line=dict(width=0)),
    hovertemplate=(
        "Area: %{hovertext}<br>"
        "Avg. Rent: $%{x:,.0f}<br>"
        "Felony Crimes: %{y:,}<extra></extra>"
    )
)

fig2.update_layout(
    template="plotly_white",
    xaxis=dict(tickprefix="$"),
    margin=dict(l=60, r=40, t=60, b=50)
)

fig2.show()

## Interactive Scatter Plot: Total Crime vs. Avg Median Rent by Area

In [ ]:
area_agg_data = (
    merged_df.groupby("precinct_area", as_index=False)
    .agg(
        total_crime=("count", "sum"),
        avg_median_rent=("median_rent", "mean"),
        borough=("Borough", "first")
    )
)

fig = px.scatter(
    area_agg_data,
    x="avg_median_rent",
    y="total_crime",
    color="borough",
    size="total_crime",                 # keep size = crime
    size_max=22,                        # cap bubble size
    hover_name="precinct_area",         # show name on hover only
    title="Precinct Areas: Total Crime Incidents vs. Average Median Rent",
    labels={
        "avg_median_rent": "Average Median Rent (USD)",
        "total_crime": "Total Crime Incidents",
        "borough": "Borough"
    },
    opacity=0.65
)

fig.update_traces(
    marker=dict(line=dict(width=0)),
    hovertemplate=(
        "Area: %{hovertext}<br>"
        "Borough: %{marker.color}<br>"
        "Avg. Rent: $%{x:,.0f}<br>"
        "Total Crimes: %{y:,}<extra></extra>"
    )
)

fig.update_layout(
    template="plotly_white",
    legend_title_text="Borough",
    margin=dict(l=70, r=40, t=70, b=60)
)

fig.update_xaxes(tickprefix="$")

fig.show()
# Export a static image for GitHub rendering
import os
os.makedirs("figs", exist_ok=True)
fig.write_image("figs/fig_04.png", scale=2)
from IPython.display import HTML
HTML(f'<img src="figs/fig_04.png" alt="Figure" width="100%"/>')


## Crime Category vs. Rent Correlation Analysis

In [ ]:
area_category_crime_rent = (
    merged_df
    .groupby(["precinct_area", "law_cat_cd"], as_index=False)
    .agg(
        total_category_crime=("count", "sum"),
        avg_median_rent=("median_rent", "mean")
    )
)

fig = px.scatter(
    area_category_crime_rent,
    x="avg_median_rent",
    y="total_category_crime",
    facet_col="law_cat_cd",
    facet_col_spacing=0.06,
    hover_name="precinct_area",
    color="law_cat_cd",
    title="Crime Incidents vs. Average Median Rent by Precinct Area (Faceted by Offense Category)",
    labels={
        "avg_median_rent": "Average Median Rent (USD)",
        "total_category_crime": "Total Crime Incidents",
        "law_cat_cd": "Offense Category"
    },
    opacity=0.65,
    color_discrete_map={
        "FELONY": "#d62728",
        "MISDEMEANOR": "#ff7f0e",
        "VIOLATION": "#2ca02c"
    }
)

# Make markers cleaner and hover more informative
fig.update_traces(
    marker=dict(size=6, line=dict(width=0)),
    hovertemplate=(
        "Area: %{hovertext}<br>"
        "Avg. Rent: $%{x:,.0f}<br>"
        "Total Crimes: %{y:,}<extra></extra>"
    )
)

# Clean facet labels + professional layout
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.update_layout(
    template="plotly_white",
    showlegend=False,  # facet already separates categories
    margin=dict(l=70, r=40, t=80, b=60)
)

fig.update_xaxes(tickprefix="$")

fig.show()
# Export a static image for GitHub rendering
import os
os.makedirs("figs", exist_ok=True)
fig.write_image("figs/fig_05.png", scale=2)
from IPython.display import HTML
HTML(f'<img src="figs/fig_05.png" alt="Figure" width="100%"/>')


# Final Dash Applicaton

## Plot 1: Monthly Trend of Crime Count (FELONY as example)

In [ ]:
# Ensure date column is datetime
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Filter to FELONY crimes only
felony_df = merged_df[merged_df["law_cat_cd"] == "FELONY"]

# Aggregate monthly FELONY crime counts
monthly_felony_trend_df = (
    felony_df
    .set_index("date")
    .resample("ME")["count"]   # Month End (future-proof)
    .sum()
    .reset_index()
)

# Create line plot
fig1 = px.line(
    monthly_felony_trend_df,
    x="date",
    y="count",
    title="Monthly Felony Crime Incidents in New York City",
    labels={
        "date": "Date",
        "count": "Total Felony Incidents"
    }
)

# Professional styling
fig1.update_traces(
    line=dict(width=2),
    hovertemplate="Month: %{x|%b %Y}<br>Felony Incidents: %{y:,}<extra></extra>"
)

fig1.update_layout(
    template="plotly_white",
    margin=dict(l=60, r=40, t=60, b=50),
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

fig1.show()

## Plot2: Crime Count by Precinct Area (FELONY as example)

In [ ]:
law_category = "FELONY"

# Aggregate felony crime by precinct area
felony_by_area_df = (
    merged_df[merged_df["law_cat_cd"] == law_category]
    .groupby("precinct_area", as_index=False)["count"]
    .sum()
    .sort_values(by="count", ascending=False)
    .head(15)   # Top 15 areas
)

fig3 = px.bar(
    felony_by_area_df,
    x="count",
    y="precinct_area",
    orientation="h",
    title="Top 15 Precinct Areas by Felony Crime Incidents",
    labels={
        "precinct_area": "Precinct Area",
        "count": "Total Felony Incidents"
    }
)

fig3.update_traces(
    hovertemplate="Area: %{y}<br>Total Felonies: %{x:,}<extra></extra>"
)

fig3.update_layout(
    template="plotly_white",
    yaxis=dict(categoryorder="total ascending"),
    margin=dict(l=140, r=40, t=60, b=50),
    height=500
)

fig3.show()

## Plot 3: Crime Count vs Median Rent by Borough (FELONY as example)

In [ ]:
felony_df = merged_df[merged_df["law_cat_cd"] == "FELONY"]

# Remove rows with missing or zero median rent, if any
felony_df = felony_df[felony_df["median_rent"].notna() & (felony_df["median_rent"] > 0)]

# Create scatter plot
fig4 = px.scatter(
    felony_df,
    x="median_rent",
    y="count",
    color="Borough",
    title="💥 FELONY Crime Count vs Median Rent by Borough",
    labels={
        "median_rent": "Median Rent ($)",
        "count": "Crime Count"
    }
)

fig4.update_traces(
    marker=dict(size=5, opacity=0.6, line=dict(width=0)),
    hovertemplate=(
        "Borough: %{marker.color}<br>"
        "Median Rent: $%{x:,.0f}<br>"
        "Felony Count: %{y}<extra></extra>"
    )
)

fig4.update_layout(
    title="Felony Crime Count vs. Median Rent by Borough",
    template="plotly_white",
    xaxis=dict(title="Median Rent (USD)", tickprefix="$"),
    yaxis=dict(title="Felony Crime Count"),
    legend_title_text="Borough",
    margin=dict(l=60, r=40, t=60, b=50)
)
fig4.show()


## Plot 4: Rent Distribution by Borough (Box Plot)

In [ ]:
valid_rent_df = merged_df[
    merged_df["median_rent"].notna() & (merged_df["median_rent"] > 0)
]

# Create box plot
fig5 = px.box(
    valid_rent_df,
    x="Borough",
    y="median_rent",
    title="Rent Distribution by Borough",
    labels={
        "Borough": "Borough",
        "median_rent": "Median Rent (USD)"
    }
)

# Keep your presentation, improve background
fig5.update_layout(
    template="plotly_white",              
    margin=dict(l=60, r=40, t=60, b=50),
    yaxis=dict(tickprefix="$")           
)

fig5.show()

##  Plot 5: Heatmap of Average Danger Ratio by Precinct Area and Month

In [ ]:
# Ensure date column is datetime
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Create month-level timestamp
merged_df["month"] = merged_df["date"].dt.to_period("M").dt.to_timestamp()

# Aggregate average danger ratio by month and precinct area
danger_trend_df = (
    merged_df
    .groupby(["month", "precinct_area"], as_index=False)["danger_ratio"]
    .mean()
    .rename(columns={"danger_ratio": "avg_danger_ratio"})
)

# Pivot for heatmap
danger_pivot = danger_trend_df.pivot(
    index="month",
    columns="precinct_area",
    values="avg_danger_ratio"
)

# Create heatmap
fig6 = px.imshow(
    danger_pivot.T,                         # precincts on y-axis
    aspect="auto",
    color_continuous_scale="Reds",
    labels=dict(
        x="Month",
        y="Precinct Area",
        color="Average Danger Ratio"
    ),
    title="Average Crime-to-Rent Danger Ratio by Precinct Area and Month"
)

# Professional layout
fig6.update_layout(
    template="plotly_white",
    margin=dict(l=100, r=40, t=70, b=60),
    xaxis=dict(tickformat="%Y-%m")
)

fig6.show()

## Plot 6: Average Danger Ratio by Precinct Area (Bar Chart)

In [ ]:
# Group by precinct_area and compute average danger ratio
avg_danger_df = (
    merged_df.groupby("precinct_area")["danger_ratio"]
    .mean()
    .reset_index(name="avg_danger_ratio")
    .sort_values(by="avg_danger_ratio", ascending=False)
)

# Create bar chart
fig7 = px.bar(
    avg_danger_df,
    x="precinct_area",
    y="avg_danger_ratio",
    title="📊 Average Danger Ratio by Precinct Area (2015-01-01 to 2024-12-01)",
    labels={"precinct_area": "Precinct Area", "avg_danger_ratio": "Average Danger Ratio"}
)

# Tweak layout to improve readability
fig7.update_layout(
    xaxis_tickangle=-45,
    margin=dict(l=60, r=40, t=60, b=160),
    height=600
)

fig7.show()


## Plot 7: Overall Median Rent Trend

In [ ]:
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Aggregate monthly average median rent (Month End)
monthly_rent_df = (
    merged_df
    .set_index("date")
    .resample("ME")["median_rent"]
    .mean()
    .reset_index()
)

# Create line plot
fig8 = px.line(
    monthly_rent_df,
    x="date",
    y="median_rent",
    title="Monthly Average Median Rent in New York City",
    labels={
        "date": "Date",
        "median_rent": "Median Rent (USD)"
    }
)

# Professional styling
fig8.update_traces(
    line=dict(width=2),
    hovertemplate="Month: %{x|%b %Y}<br>Median Rent: $%{y:,.0f}<extra></extra>"
)

fig8.update_layout(
    template="plotly_white",
    margin=dict(l=60, r=40, t=60, b=50),
    yaxis=dict(tickprefix="$"),
    xaxis=dict(showgrid=True),
    yaxis_showgrid=True
)

fig8.show()

## Plot 8: Overall Average Danger Ratio Trend (Monthly)

In [ ]:
merged_df["date"] = pd.to_datetime(merged_df["date"])

# Aggregate monthly average danger ratio (Month End)
monthly_danger_df = (
    merged_df
    .set_index("date")
    .resample("ME")["danger_ratio"]
    .mean()
    .reset_index()
)

# Create line plot
fig9 = px.line(
    monthly_danger_df,
    x="date",
    y="danger_ratio",
    title="Monthly Average Crime-to-Rent Danger Ratio",
    labels={
        "date": "Date",
        "danger_ratio": "Average Danger Ratio"
    }
)

# Professional styling
fig9.update_traces(
    line=dict(width=2),
    hovertemplate=(
        "Month: %{x|%b %Y}<br>"
        "Avg. Danger Ratio: %{y:.3f}<extra></extra>"
    )
)

fig9.update_layout(
    template="plotly_white",
    margin=dict(l=60, r=40, t=60, b=50),
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

fig9.show()

## Plot 10: Latest Median Rent by ZIP Code (Choropleth Map)

In [ ]:
# --- Prep ---
merged_df["date"] = pd.to_datetime(merged_df["date"], errors="coerce")

zip_flat = (
    merged_df
    .explode("ZIP Codes")
    .rename(columns={"ZIP Codes": "ZIP"})
    .dropna(subset=["ZIP", "date", "median_rent"])
    .copy()
)

zip_flat["ZIP"] = (
    zip_flat["ZIP"]
    .astype(str)
    .str.extract(r"(\d+)")[0]
    .str.zfill(5)
)

zip_flat["median_rent"] = pd.to_numeric(zip_flat["median_rent"], errors="coerce")
zip_flat = zip_flat.dropna(subset=["median_rent"])

# Latest rent per ZIP
idx = zip_flat.groupby("ZIP")["date"].idxmax()
latest_zip_rent = zip_flat.loc[idx, ["ZIP", "date", "median_rent"]]

# GeoJSON
geojson_url = (
    "https://raw.githubusercontent.com/OpenDataDE/"
    "State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json"
)

# Map
fig = px.choropleth_map(
    latest_zip_rent,
    geojson=geojson_url,
    locations="ZIP",
    featureidkey="properties.ZCTA5CE10",
    color="median_rent",
    color_continuous_scale="Blues",
    center={"lat": 40.7128, "lon": -74.0060},
    zoom=9.8,
    opacity=0.75,
    labels={"median_rent": "Median Rent (USD)"},
    title="Latest Median Rent by ZIP Code (NYC)"
)

# Professional hover & layout
fig.update_traces(
    hovertemplate=(
        "ZIP: %{location}<br>"
        "As of: %{customdata[0]|%b %Y}<br>"
        "Median Rent: $%{z:,.0f}<extra></extra>"
    ),
    customdata=latest_zip_rent[["date"]].to_numpy()
)

fig.update_layout(
    template="plotly_white",
    margin=dict(l=0, r=0, t=60, b=0),
    title=dict(x=0.5),
    coloraxis_colorbar=dict(
        title="Median Rent (USD)",
        tickprefix="$"
    )
)

fig.show()
# Export a static image for GitHub rendering
import os
os.makedirs("figs", exist_ok=True)
fig.write_image("figs/fig_06.png", scale=2)
from IPython.display import HTML
HTML(f'<img src="figs/fig_06.png" alt="Figure" width="100%"/>')


In [ ]:
print('Done')